In [1]:
# Note to import from .py files, must follow structure
# from <.py filename excluding '.py'> import <class name>
# Optionslam creds: aspringfastlaner Options2018

# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import sched, time
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.plotly as py
import plotly
import statsmodels.api as sm
from scipy.stats import skewnorm as skn
from scipy.stats import norm
import statsmodels.api as sm
import plotly.graph_objs as go

C:\Users\Fang\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



In [29]:
def optionslam_scrape(ticker):
    site = 'https://www.optionslam.com/earnings/stocks/' + ticker
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")
    soup = soup.prettify()
    earnings_dict = {'Ticker': ticker}
    
    # Check if there's weekly options
    curr7_implied = "Current 7 Day Implied Movement:"
    implied_move_weekly = "Implied Move Weekly:"
    nextearnings = "Next Earnings Date:"
    if curr7_implied not in soup:
        return False
    
    # Parsing if weekly options exist
    # Next earnings date and before or after
    earnings_start_string = "Next Earnings Date:"
    earnings_end_string = '</font>'
    raw_earnings_string = (soup.split(earnings_start_string))[1].split(earnings_end_string)[0].replace('\n','').strip()
    earnings_date = str((raw_earnings_string.split('<b>'))[1].split('<font size="-1">')).split("'")[1].strip()
    earnings_time = str(raw_earnings_string[-2:].strip()).strip()
    
    earnings_dict['Date'] = earnings_date
    earnings_dict['Earnings Time'] = earnings_time
    
    # Parsing 7 day implied move if weekly option exists
    ending_string = '<font size="-2">'
    curr_7 = (soup.split(curr7_implied))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    earnings_dict['Current 7 Day Implied'] = curr_7
    
    # Parsing Weekly Implied move if weekly option exists
    if implied_move_weekly in soup:
        weekly_implied = (soup.split(implied_move_weekly))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    else:
        weekly_implied = ''
    earnings_dict["Implied Move Weekly"] = weekly_implied
    
    return earnings_dict

In [33]:
# Yahoo Earnings Calendar Check
yahoo_url = 'https://finance.yahoo.com/calendar/earnings?day='
today = dt.datetime.now().strftime('%Y-%m-%d')
tomorrow = (dt.datetime.now() + dt.timedelta(days = 1)).strftime('%Y-%m-%d')


'2018-02-24'

In [99]:
testsite = 'https://finance.yahoo.com/calendar/earnings?day=2018-02-26&offset=100&size=100'
res = requests.get(testsite)
soup = bs(requests.get(testsite).text, "lxml")
table = soup.find_all('table')[0]

IndexError: list index out of range

In [96]:
# Looping through the soup lxml text table format
# and splitting each row as a individual string
# and parsing string to retrieve the date,
# open, and close information.
i = 1
end_row = 3
for row in table.find_all('tr'):
    # Individual row stores current row item and delimits on '\n'
    individual_row = str(row).split('\n')
    row_items = individual_row[0].split('</span>')[:3]
    # row_items is parsed string for each current row where each
    # item in list is the date, open, high, low, close, and volume
    if i == 1:
        headers = row_items
    
    elif i == end_row:
        break
    
    i += 1

In [91]:
(dt.datetime.now() + dt.timedelta(days = 3)).strftime('%a')

'Tue'

In [97]:
row_items

['<tr class="data-rowJOBS Bgc($extraLightBlue):h BdT Bdc($tableBorderGray) Bdc($tableBorderBlue):h H(33px) Bgc($altRowColor) Whs(nw)" data-reactid="50"><td class="data-col0 Ta(start) Pstart(15px) Pend(15px) W(10%)" data-reactid="51"><a class="Fw(b)" data-reactid="52" data-symbol="JOBS" href="/quote/JOBS?p=JOBS" title="51job Inc">JOBS</a></td><td class="data-col1 Ta(start) Pend(10px)" data-reactid="53">51job Inc</td><td class="data-col2 Ta(end) Pstart(15px) W(20%)" data-reactid="54"><span data-reactid="55">After Market Close',
 '</td><td class="data-col3 Ta(end) Pstart(15px) W(10%)" data-reactid="56">0.64</td><td class="data-col4 Ta(end) Pstart(15px) W(10%)" data-reactid="57">-</td><td class="data-col5 Ta(end) Pend(15px) Pstart(15px) W(13%)" data-reactid="58"><span class="" data-reactid="59">-',
 '</td></tr>']